<a href="https://colab.research.google.com/github/Ka16rt11hi05cK/Stock_Price_Prediction/blob/main/Stock_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance
import yfinance as yf
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from datetime import date, timedelta

def predict_tomorrows_stock_movement(ticker_symbol='AAPL', start_date='2020-01-01'):
    try:
        end_date = date.today()

        stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)

        if stock_data.empty:
            print(f"No data found for symbol {ticker_symbol}. Please check the ticker symbol and date range.")
            return

        stock_data['ma50'] = stock_data['Close'].rolling(window=50).mean()
        stock_data['ma200'] = stock_data['Close'].rolling(window=200).mean()
        stock_data['target'] = (stock_data['Close'].shift(-1) > stock_data['Close']).astype(int)

        processed_data = stock_data.dropna()

        features = ['Open', 'High', 'Low', 'Close', 'Volume', 'ma50', 'ma200']
        X_train = processed_data[features]
        y_train = processed_data['target']

        model = RandomForestClassifier(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)

        latest_data = stock_data[features].iloc[[-1]]

        if latest_data.isnull().values.any():
            print(f"Could not make a prediction for {ticker_symbol} as the latest data contains NaN values (likely due to insufficient history for moving averages).")
            print("Try an earlier start_date.")
            return

        prediction = model.predict(latest_data)
        prediction_proba = model.predict_proba(latest_data)

        tomorrow = end_date + timedelta(days=1)
        print(f"--- Prediction for {ticker_symbol} on {tomorrow.strftime('%Y-%m-%d')} ---")

        if prediction[0] == 1:
            confidence = prediction_proba[0][1]
            print(f"Prediction: UP \nConfidence: {confidence:.2%}")
        else:
            confidence = prediction_proba[0][0]
            print(f"Prediction: DOWN \nConfidence: {confidence:.2%}")

    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == '__main__':
    predict_tomorrows_stock_movement(ticker_symbol='NIFTY_BANK', start_date='2019-01-01')



<ipython-input-8-3550102420>:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)
ERROR:yfinance:HTTP Error 401: 
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['NIFTY_BANK']: YFTzMissingError('possibly delisted; no timezone found')


No data found for symbol NIFTY_BANK. Please check the ticker symbol and date range.
